In [ ]:
# 导入库
import akshare as ak
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# akshare 版本  
print(ak.__version__)

In [ ]:
# 获取白酒行业成分股数据

stocks = ak.stock_board_industry_cons_ths(symbol="白酒")
stocks.to_csv("白酒股票.csv")


In [ ]:
# 获取板块股票逐年利润表
profit_y_dict = dict()

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    if (code.startswith('00') != True):
        market ='sh'
    symbol = "{}{}".format(market, code)
    profit_y = ak.stock_profit_sheet_by_yearly_em(symbol=symbol)
    # profit_y.to_csv("profit_{}.csv".format(code))

    profit_y_dict[code] = profit_y

# print(profit_y_dict)

In [ ]:
# 获取板块股票归属母公司利润表

profit_dict = {}
for i in tqdm(range(len(stocks))):
    code = stocks.loc[i, "代码"]

    p = pd.Series(list(profit_y_dict[code]["PARENT_NETPROFIT"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]).dt.date)
    profit_dict[code] = p

df_parent_profit = pd.DataFrame(profit_dict)
# df_parent_profit.fillna(0, inplace=True)

In [ ]:
df_parent_profit['000858']

In [ ]:
df_parent_profit['000858'].to_csv("profit_000858.csv")

In [ ]:
df_eps = pd.DataFrame()
eps_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    
    s1 = pd.Series(list(profit_y_dict[code]["BASIC_EPS"]), index=pd.to_datetime(profit_y_dict[code]["REPORT_DATE"]))
    eps_dict["{}-{}".format(code, name)] = s1

df_eps = pd.DataFrame(eps_dict)
df_eps.fillna(0, inplace=True)

In [ ]:
df_eps.to_csv("eps.csv")

In [ ]:
df_eps.fillna(0, inplace=True)


In [ ]:
print(type(df_eps.index))

In [ ]:
df_eps['2010':]

In [ ]:
df_close

In [ ]:
df_eps['2010':].plot(kind='bar')

In [ ]:
print(df_eps.mean().sort_values(ascending=False))

In [ ]:
df_eps.sort_index(ascending=True).plot(kind="bar", colormap="cubehelix",legend=False)

In [ ]:
### 导出每股收益到csv
profiet_y = ak.stock_profit_sheet_by_yearly_em(symbol="sh603919")
print(profit_y["BASIC_EPS"][0:5])
profiet_y.to_csv("profit_603919.csv")

In [ ]:
df_stocks.sort_index(ascending=True).pct_change().plot(kind="bar", colormap="cubehelix")

In [ ]:
import datetime

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
print(all_trade_date)

ts = pd.Series(np.random.randn(len(trade_date_index)), index=trade_date_index)

enddate = datetime.datetime.now().strftime("%Y%m%d")
print(ts)
print(ts['2016-01-01':enddate])

In [ ]:
# 导出白酒成分股到csv
stocks.to_csv("bj.csv")

In [ ]:
#股票走势
import datetime

startdate = datetime.date(2000,1,1)
enddate = datetime.datetime.now().strftime("%Y%m%d")

trade_date_list= ak.tool_trade_date_hist_sina()
all_trade_date = pd.to_datetime(trade_date_list["trade_date"])
all_trade_date.index = all_trade_date
trade_date_index = all_trade_date[startdate:enddate]

# df_close = pd.DataFrame()
cl_dict = {}

for i in tqdm(range(len(stocks))):
# for i in range(3):
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]
    stock_hist = ak.stock_zh_a_hist(symbol=code, period="daily", start_date=startdate.strftime("%Y%m%d"), end_date=enddate, adjust="hfq")
    
    stock_hist.to_csv(f"{code}-stock.csv")

    cl = stock_hist["收盘"]
    # if (len(cl) != len(trade_date_index)):
    #     print("股票数据长度不匹配:{}:{}-{}".format(len(cl), len(trade_date_index), code))
    #     continue

    ts = pd.Series(list(cl), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_high = pd.Series(list(stock_hist["最高"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    ts_low = pd.Series(list(stock_hist["最低"]), index=pd.to_datetime(stock_hist["日期"]).dt.date)
    # print(ts)
    # print(stock_hist["收盘"])
    cl_dict["{}".format(code)] = ts
df_close = pd.DataFrame(cl_dict)
  
# df_close.plot(subplots=True, figsize=(16,16))
# print(stock_hist)

In [ ]:
stocks

In [ ]:
ret = (1 + df_close.pct_change()).cumprod()
print(ret)
# ret.to_csv("ret.csv")
# df_close.to_csv("cl.csv")

# print(df_close.loc["2016-01-29",  :])
# print(df_close.resample('M').asfreq())

In [ ]:
ret.plot(kind="line")

In [ ]:
print(df_close['600519'].resample('M').last())


In [ ]:
print(df_close)

In [ ]:
### 月收益率
month_ret = df_close.pct_change().resample('M').apply(lambda x : ((1+x).prod() - 1))
print(month_ret)

In [ ]:
### 年度收益率
year_ret = df_close.pct_change().resample('Y').apply(lambda x : ((1+x).prod() - 1))
print(year_ret)
# year_ret.to_csv('year_ret.csv')

In [ ]:
### 作图区

ax1 = df_parent_profit['000858'].dropna().sort_index().plot(kind="bar")

pc = df_parent_profit['000858'].dropna().sort_index().pct_change()
pc.fillna(0, inplace=True)

ax2 = ax1.twinx()
ax2.set_ylabel('change rate')
ax2.set_ylim(-1,1)
pc.plot(kind='line', marker='*', color='red', ms=10)

In [ ]:
year_ret['000858'].plot(kind='bar')

In [ ]:
df_dividents = pd.DataFrame()
dividents_dict = {}

for i in tqdm(range(len(stocks))):
    market = 'sz'
    code = stocks.loc[i, "代码"]
    name = stocks.loc[i, "名称"]

    dividents = ak.stock_dividents_cninfo(symbol=code)
    # sg = pd.Series(list(dividents["送股比例"]), index=pd.to_datetime(dividents["日期"]))
    # zz = pd.Series(list(dividents["转增比例"]), index=pd.to_datetime(dividents["日期"]))
    dividents.to_csv(f"{code}-div.csv")
    px = pd.Series(list(dividents["派息比例"]), index=pd.to_datetime(dividents["报告时间"]))

    dividents_dict["{}-{}".format(code, name)] = px

df_dividents = pd.DataFrame(dividents_dict)
df_dividents.fillna(0, inplace=True)

In [44]:
import akshare as ak
from tqdm import tqdm 
import pandas as pd


def balance_sheet_analyze(industry : str, name: str):
    stocks = ak.stock_board_industry_cons_ths(symbol=industry)
    index = ['货币资金', '未分配利润', '应收票据及应收账款', '存货', '流动资产合计', '所有者权益']
    columns =  []
    # date_index = None
    report_all = pd.DataFrame()

    for i in range(len(stocks)):
        code = stocks.loc[i, "代码"]
        columns.append(code)

    for i in tqdm(range(len(stocks))):
        code = stocks.loc[i, "代码"]

        balance = ak.stock_financial_report_sina(stock=code, symbol=name)
        # if date_index is None:
            # print(pd.to_datetime(balance["报表日期"]))
        date_index = pd.to_datetime(balance["报表日期"])

        # if report_all is None:
        #     report_all = pd.DataFrame(
        #         # np.random.randn(204,1),
        #         columns=columns,
        #         index=pd.MultiIndex.from_product(
        #             [date_index, index]
        #         )
        #     )            
        report = pd.DataFrame(
            # np.random.randn(204,1),
            columns=[code],
            index=pd.MultiIndex.from_product(
                [date_index, index]
            )
        )   #             

        for idx in range(len(index)):
            # tmpDf = pd.DataFrame(balance[index[idx]], index=pd.to_datetime(balance["报表日期"]))
            report.loc[(slice(None), index[idx]), :] = balance[index[idx]].values
        # print(report_all)
        report_all = pd.concat([report_all, report], axis=1)
        # print(report_all)

    return report_all

In [45]:
df = balance_sheet_analyze("白酒", "资产负债表")
print(df)

  5%|▌         | 1/20 [00:00<00:03,  5.69it/s]

                             603198
报表日期                               
2022-09-30 货币资金        740288441.47
           未分配利润      3965105194.33
           应收票据及应收账款    49099234.83
           存货         3865769613.45
           流动资产合计     6709997209.54
...                             ...
2011-12-31 未分配利润       491702636.04
           应收票据及应收账款              0
           存货          874195097.39
           流动资产合计     2095310335.48
           所有者权益                NaN

[204 rows x 1 columns]


 10%|█         | 2/20 [00:00<00:04,  3.90it/s]

                             603198         600199
报表日期                                              
2022-09-30 货币资金        740288441.47   488621503.56
           未分配利润      3965105194.33   533801064.49
           应收票据及应收账款    49099234.83   150964553.55
           存货         3865769613.45  1310102797.28
           流动资产合计     6709997209.54  2092156884.70
...                             ...            ...
1995-12-31 未分配利润                NaN              0
           应收票据及应收账款            NaN              0
           存货                   NaN    71495879.79
           流动资产合计               NaN   157626639.22
           所有者权益                NaN            NaN

[564 rows x 2 columns]


 15%|█▌        | 3/20 [00:00<00:04,  3.58it/s]

...

                             603198         600199          000568  \
报表日期                                                                 
2022-09-30 货币资金        740288441.47   488621503.56  13684458440.06   
           未分配利润      3965105194.33   533801064.49  24631260469.26   
           应收票据及应收账款    49099234.83   150964553.55      2635634.68   
           存货         3865769613.45  1310102797.28   8977145505.91   
           流动资产合计     6709997209.54  2092156884.70  26784348364.71   
...                             ...            ...             ...   
2001-03-31 未分配利润                NaN            NaN             NaN   
           应收票据及应收账款            NaN            NaN             NaN   
           存货                   NaN            NaN             NaN   
           流动资产合计               NaN            NaN             NaN   
           所有者权益                NaN            NaN             NaN   

                             000799          603369         600559  \
报表日期               

 95%|█████████▌| 19/20 [00:04<00:00,  4.15it/s]

                             603198         600199          000568  \
报表日期                                                                 
2022-09-30 货币资金        740288441.47   488621503.56  13684458440.06   
           未分配利润      3965105194.33   533801064.49  24631260469.26   
           应收票据及应收账款    49099234.83   150964553.55      2635634.68   
           存货         3865769613.45  1310102797.28   8977145505.91   
           流动资产合计     6709997209.54  2092156884.70  26784348364.71   
...                             ...            ...             ...   
2001-03-31 未分配利润                NaN            NaN             NaN   
           应收票据及应收账款            NaN            NaN             NaN   
           存货                   NaN            NaN             NaN   
           流动资产合计               NaN            NaN             NaN   
           所有者权益                NaN            NaN             NaN   

                             000799          603369         600559  \
报表日期               

100%|██████████| 20/20 [00:04<00:00,  4.05it/s]

                             603198         600199          000568  \
报表日期                                                                 
2022-09-30 货币资金        740288441.47   488621503.56  13684458440.06   
           未分配利润      3965105194.33   533801064.49  24631260469.26   
           应收票据及应收账款    49099234.83   150964553.55      2635634.68   
           存货         3865769613.45  1310102797.28   8977145505.91   
           流动资产合计     6709997209.54  2092156884.70  26784348364.71   
...                             ...            ...             ...   
2001-03-31 未分配利润                NaN            NaN             NaN   
           应收票据及应收账款            NaN            NaN             NaN   
           存货                   NaN            NaN             NaN   
           流动资产合计               NaN            NaN             NaN   
           所有者权益                NaN            NaN             NaN   

                             000799          603369         600559  \
报表日期               

In [34]:
stocks = ak.stock_board_industry_cons_ths(symbol="白酒")
index = ['货币资金', '未分配利润', '应收票据及应收账款', '存货', '流动资产合计', '所有者权益']
columns =  []
# date_index = None
report_all = pd.DataFrame()

for i in range(len(stocks)):
    code = stocks.loc[i, "代码"]
    columns.append(code)

# for i in tqdm(range(len(stocks))):
code = stocks.loc[i, "代码"]

balance = ak.stock_financial_report_sina(stock=code, symbol="资产负债表")
# if date_index is None:
    # print(pd.to_datetime(balance["报表日期"]))
date_index = pd.to_datetime(balance["报表日期"])

# if report_all is None:
#     report_all = pd.DataFrame(
#         # np.random.randn(204,1),
#         columns=columns,
#         index=pd.MultiIndex.from_product(
#             [date_index, index]
#         )
#     )            
report = pd.DataFrame(
    # np.random.randn(204,1),
    # balance['货币资金'].values,
    columns=[code],
    index=pd.MultiIndex.from_product(
        [date_index, index]
    )
)   #             

# for idx in range(len(index)):
    # tmpDf = pd.DataFrame(balance[index[idx]], index=pd.to_datetime(balance["报表日期"]))
# print(balance['货币资金'])
print(report)
# print(report.loc[(slice(None), '货币资金'), :] )
# print(balance['货币资金'])
report.loc[(slice(None), '货币资金'), :] = balance['货币资金'].values
print(report.loc[(slice(None), '货币资金'), :] )

# print(report.loc[(slice(None), '货币资金'), :])
# print(report_all)
# report_all = pd.concat([report_all, report], axis=1)
# print(report_all)

                     603589
报表日期                       
2022-09-30 货币资金         NaN
           未分配利润        NaN
           应收票据及应收账款    NaN
           存货           NaN
           流动资产合计       NaN
...                     ...
2011-12-31 未分配利润        NaN
           应收票据及应收账款    NaN
           存货           NaN
           流动资产合计       NaN
           所有者权益        NaN

[204 rows x 1 columns]
                        603589
报表日期                          
2022-09-30 货币资金   912932078.12
2022-06-30 货币资金   728565719.20
2022-03-31 货币资金   859569587.04
2021-12-31 货币资金  1881057555.44
2021-09-30 货币资金  1231287620.45
2021-06-30 货币资金   926767447.02
2021-03-31 货币资金  1251333830.92
2020-12-31 货币资金  1504325061.27
2020-09-30 货币资金  1307374588.05
2020-06-30 货币资金  1219297057.92
2020-03-31 货币资金  1423586190.39
2019-12-31 货币资金  1533192630.12
2019-09-30 货币资金  1137729583.80
2019-06-30 货币资金  1364763811.12
2019-03-31 货币资金  1319891058.00
2018-12-31 货币资金  1271782193.76
2018-09-30 货币资金   710749758.10
2018-06-30 货币资金   92927